# Create and update the AOI lookup table

In this notebook, we are going to create a lookup table containing the name and id of all the geometries with precalculated data used in the Areas Of Interest section (AOI) of the Half-Earth map. The purpose of the AOI table is to enrich and improve the search functionality of the AOIs, allowing the users to search among all features with precalculated data. 

In July 2022 these include: national boundaries, subnational boundaries and protected areas. Because the *Places for the Half-Earth Future* do not have names, they will be excluded from this table. 

Here, we are going to create the AOI lookup table and publish it in AGOL. To integrate future changes, the table will be updated using a notebook hosted in ArcGIS Online, but in the present notebook we'll provide an example of how the update is done. 

## Table of contents
1. [First steps](#first)
    1. [Import packages](#packages)
    2. [Connect to ESRI](#esri)
    3. [Utils](#utils)
    4. [Import AOI layers and tables](#layers)
2. [Create AOI lookup table](#table)
    1. [National boundaries](#national)
    2. [Subnational boundaries](#subnational)
    3. [Protected areas](#protected)
    4. [AOI lookup table](#lookup)
3. [Publish AOI lookup table](#publish)
4. [Update AOI lookup table: Example](#update)



---
<a id='first'></a>
## First steps

<a id='packages'></a>
### Import packages

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

<a id='esri'></a>
### Connect to ESRI

In [2]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [3]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [4]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


<a id='utils'></a>
### Utils

In [5]:
# Get gadm0 hosted layer as sdf (only desired fields)
def getGadm0fromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.layers[0]
    sdf = flayer.query(out_fields=['NAME_0', 'MOL_ID', 'alternative_names',  'NAME_0_es', 'NAME_0_fr', 'NAME_0_pt']).sdf
    return sdf

In [6]:
# Get gadm1 hosted layer as sdf (only desired fields)
def getGadm1fromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.layers[0]
    sdf = flayer.query(out_fields=['NAME_1', 'MOL_ID']).sdf
    return sdf

In [7]:
# Get wdpa hosted table as sdf (only desired fields)
def getPAfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query(out_fields=['NAME', 'MOL_ID']).sdf
    return sdf

<a id='layers'></a>
### Import AOI layers and tables

We need to select only the columns that we need because trying to import the entire tables as dataframes can cause timeouts in big services such as gadm1 or wdpa.

In [8]:
gadm0 = getGadm0fromId('7b940493af104f7e86f1844b432bf5d3')
gadm0.head()

,ObjectId,NAME_0,MOL_ID,alternative_names,NAME_0_es,NAME_0_fr,NAME_0_pt,SHAPE
0,1,Aruba,1,None,Aruba,Aruba,Aruba,"{""rings"": [[[-7789937.05527832, 1399228.874996..."
1,2,Afghanistan,2,None,Afganistán,Afghanistan,Afeganistão,"{""rings"": [[[7628327.94472602, 3731107.8519135..."
2,3,Angola,3,None,Angola,Angola,Angola,"{""rings"": [[[1306164.11380903, -1882741.235359..."
3,4,Anguilla,4,None,Anguila,Anguilla,Anguila,"{""rings"": [[[-7060299.75801276, 2106609.789138..."
4,5,Åland,5,None,Åland,Åland,Ilhas Åland,"{""rings"": [[[2373548.29389075, 8344256.0518993..."


In [9]:
gadm1 = getGadm1fromId('fcb5fdda4a8a454b8b1c9545c18b4d91')
gadm1.head()

,ObjectId,NAME_1,MOL_ID,SHAPE
0,1,Zhemgang,392,"{""rings"": [[[10138200.6173403, 3107465.3230726..."
1,2,Central,393,"{""rings"": [[[2997865.13330869, -2723886.108033..."
2,3,Kalimantan Barat,1188,"{""rings"": [[[12267884.8526905, -341650.2280545..."
3,4,Beirut,1599,"{""rings"": [[[3953928.65381094, 4012060.0800596..."
4,5,Akkar,1597,"{""rings"": [[[3979487.49751016, 4061024.2662308..."


In [10]:
wdpa = getPAfromId('2a6207c6363b4811b4972d766b6fa23c') 
wdpa.head()

,ObjectId,NAME,MOL_ID
0,1,Thomas Hunt & Son,101
1,2,Brookdale,51
2,3,Mt Benger,151
3,4,Boulder Beach,1
4,5,Cable Bay Farms,102


----
<a id='table'></a>
## Create AOI lookup table

<a id='national'></a>
### National boundaries

Countries with alternative names (saved in *alternative_names* column) will be included in the AOI lookup table as new entries with the MOL_ID of the original name.  

In [11]:
gadm0.columns

Index(['ObjectId', 'NAME_0', 'MOL_ID', 'alternative_names', 'NAME_0_es',
       'NAME_0_fr', 'NAME_0_pt', 'SHAPE'],
      dtype='object')

In [12]:
# Get names and MOL_ID of the countries and add LANGUAGE field
names0 = gadm0[['NAME_0', 'MOL_ID']].copy()
names0['LANGUAGES'] = 'en'
names0

,NAME_0,MOL_ID,LANGUAGES
0,Aruba,1,en
1,Afghanistan,2,en
2,Angola,3,en
3,Anguilla,4,en
4,Åland,5,en
...,...,...,...
250,Spratly Islands,251,en
251,Yemen,252,en
252,South Africa,253,en
253,Zambia,254,en


In [13]:
# Get the alternative names of the countries that have them
alternative = gadm0[['alternative_names', 'MOL_ID']].dropna()
alternative# Create table for alternative names

,alternative_names,MOL_ID
44,Ivory Coast,45
51,Cape Verde,52
58,Czech Republic,59
79,UK,80
144,Republic of Macedonia,145
147,Burma,148
211,Eswatini,212
222,East Timor,223
226,Türkiye,227
234,"US, United States of America, USA",235


In [14]:
len(alternative)

10

In [15]:
# Convert string with commas into list of string and strip spaces
alternative['alternative_names'] = alternative['alternative_names'].str.split(',').map(lambda elements: [e.strip() for e in elements])

# Explode lists in the column 'alternative_names' into separate values and reset index 
alternative = alternative.explode('alternative_names').reset_index(drop=True)
alternative

,alternative_names,MOL_ID
0,Ivory Coast,45
1,Cape Verde,52
2,Czech Republic,59
3,UK,80
4,Republic of Macedonia,145
5,Burma,148
6,Eswatini,212
7,East Timor,223
8,Türkiye,227
9,US,235


In [16]:
# Change name to NAME_0
alternative = alternative.rename(columns={'alternative_names':'NAME_0'})
alternative

,NAME_0,MOL_ID
0,Ivory Coast,45
1,Cape Verde,52
2,Czech Republic,59
3,UK,80
4,Republic of Macedonia,145
5,Burma,148
6,Eswatini,212
7,East Timor,223
8,Türkiye,227
9,US,235


In [17]:
# Add language
alternative ['LANGUAGES'] = 'en'
alternative

,NAME_0,MOL_ID,LANGUAGES
0,Ivory Coast,45,en
1,Cape Verde,52,en
2,Czech Republic,59,en
3,UK,80,en
4,Republic of Macedonia,145,en
5,Burma,148,en
6,Eswatini,212,en
7,East Timor,223,en
8,Türkiye,227,en
9,US,235,en


In [18]:
# Create table with the gadm0 names
names0 = pd.concat([names0, alternative], axis=0).sort_values(by=['MOL_ID']).reset_index(drop=True).rename(columns={'NAME_0':'NAME'})
names0

,NAME,MOL_ID,LANGUAGES
0,Aruba,1,en
1,Afghanistan,2,en
2,Angola,3,en
3,Anguilla,4,en
4,Åland,5,en
...,...,...,...
262,Spratly Islands,251,en
263,Yemen,252,en
264,South Africa,253,en
265,Zambia,254,en


In [19]:
# Create tables for languages
es = gadm0[['MOL_ID', 'NAME_0_es']].rename(columns={'NAME_0_es':'NAME'})
es['LANGUAGES'] = 'es'
fr = gadm0[['MOL_ID', 'NAME_0_fr']].rename(columns={'NAME_0_fr':'NAME'})
fr['LANGUAGES'] = 'fr'
pt = gadm0[['MOL_ID', 'NAME_0_pt']].rename(columns={'NAME_0_pt':'NAME'})
pt['LANGUAGES'] = 'pt'

In [20]:
# Add translations to country table
names0 = pd.concat([names0, es], axis=0).sort_values(by=['MOL_ID']).reset_index(drop=True)
names0 = pd.concat([names0, fr], axis=0).sort_values(by=['MOL_ID']).reset_index(drop=True)
names0 = pd.concat([names0, pt], axis=0).sort_values(by=['MOL_ID']).reset_index(drop=True)
names0

,NAME,MOL_ID,LANGUAGES
0,Aruba,1,en
1,Aruba,1,fr
2,Aruba,1,pt
3,Aruba,1,es
4,Afghanistan,2,en
...,...,...,...
1027,Zâmbia,254,pt
1028,Zimbabwe,255,fr
1029,Zimbabue,255,es
1030,Zimbabwe,255,en


In [21]:
# Add layerslug for gadm0 
layerslug0 = "gadm-0-admin-areas-feature-layer"
names0['LAYERSLUG'] = layerslug0
names0

,NAME,MOL_ID,LANGUAGES,LAYERSLUG
0,Aruba,1,en,gadm-0-admin-areas-feature-layer
1,Aruba,1,fr,gadm-0-admin-areas-feature-layer
2,Aruba,1,pt,gadm-0-admin-areas-feature-layer
3,Aruba,1,es,gadm-0-admin-areas-feature-layer
4,Afghanistan,2,en,gadm-0-admin-areas-feature-layer
...,...,...,...,...
1027,Zâmbia,254,pt,gadm-0-admin-areas-feature-layer
1028,Zimbabwe,255,fr,gadm-0-admin-areas-feature-layer
1029,Zimbabue,255,es,gadm-0-admin-areas-feature-layer
1030,Zimbabwe,255,en,gadm-0-admin-areas-feature-layer


<a id='subnational'></a>
### Subnational boundaries

In [22]:
# Get names and MOL_ID of the subnational boundaries
names1 = gadm1[['NAME_1', 'MOL_ID']].rename(columns = {'NAME_1':'NAME'})
names1 

,NAME,MOL_ID
0,Zhemgang,392
1,Central,393
2,Kalimantan Barat,1188
3,Beirut,1599
4,Akkar,1597
...,...,...
3605,Peleliu,2376
3606,Sonsorol,2377
3607,Bougainville,2378
3608,Central,2379


In [23]:
# Add language
names1['LANGUAGES'] = 'en'
# Add layerslug for gadm1
layerslug1 = "gadm-1-admin-areas-feature-layer"
names1['LAYERSLUG'] = layerslug1
names1

,NAME,MOL_ID,LANGUAGES,LAYERSLUG
0,Zhemgang,392,en,gadm-1-admin-areas-feature-layer
1,Central,393,en,gadm-1-admin-areas-feature-layer
2,Kalimantan Barat,1188,en,gadm-1-admin-areas-feature-layer
3,Beirut,1599,en,gadm-1-admin-areas-feature-layer
4,Akkar,1597,en,gadm-1-admin-areas-feature-layer
...,...,...,...,...
3605,Peleliu,2376,en,gadm-1-admin-areas-feature-layer
3606,Sonsorol,2377,en,gadm-1-admin-areas-feature-layer
3607,Bougainville,2378,en,gadm-1-admin-areas-feature-layer
3608,Central,2379,en,gadm-1-admin-areas-feature-layer


<a id='protected'></a>
### Protected areas

In [20]:
wdpa.columns

Index(['ObjectId', 'NAME', 'MOL_ID'], dtype='object')

In [24]:
wdpa = wdpa[['NAME', 'MOL_ID']]
wdpa

,NAME,MOL_ID
0,Thomas Hunt & Son,101
1,Brookdale,51
2,Mt Benger,151
3,Boulder Beach,1
4,Cable Bay Farms,102
...,...,...
217481,Ostrov Starichkov,217312
217482,Vuata Ono,217313
217483,Makogai Island,217314
217484,Tubli Bay,217315


In [25]:
# Add language
wdpa['LANGUAGES'] = 'en'
# Add layerslug for gadm1
layerslug2 = "wdpa-oecm-feature-data-layer"
wdpa['LAYERSLUG'] = layerslug2
wdpa

,NAME,MOL_ID,LANGUAGES,LAYERSLUG
0,Thomas Hunt & Son,101,en,wdpa-oecm-feature-data-layer
1,Brookdale,51,en,wdpa-oecm-feature-data-layer
2,Mt Benger,151,en,wdpa-oecm-feature-data-layer
3,Boulder Beach,1,en,wdpa-oecm-feature-data-layer
4,Cable Bay Farms,102,en,wdpa-oecm-feature-data-layer
...,...,...,...,...
217481,Ostrov Starichkov,217312,en,wdpa-oecm-feature-data-layer
217482,Vuata Ono,217313,en,wdpa-oecm-feature-data-layer
217483,Makogai Island,217314,en,wdpa-oecm-feature-data-layer
217484,Tubli Bay,217315,en,wdpa-oecm-feature-data-layer


<a id='lookup'></a>
### AOI Lookup table

In [26]:
frames = [names0, names1, wdpa]
lookup = pd.concat(frames)
lookup

,NAME,MOL_ID,LANGUAGES,LAYERSLUG
0,Aruba,1,en,gadm-0-admin-areas-feature-layer
1,Aruba,1,fr,gadm-0-admin-areas-feature-layer
2,Aruba,1,pt,gadm-0-admin-areas-feature-layer
3,Aruba,1,es,gadm-0-admin-areas-feature-layer
4,Afghanistan,2,en,gadm-0-admin-areas-feature-layer
...,...,...,...,...
217481,Ostrov Starichkov,217312,en,wdpa-oecm-feature-data-layer
217482,Vuata Ono,217313,en,wdpa-oecm-feature-data-layer
217483,Makogai Island,217314,en,wdpa-oecm-feature-data-layer
217484,Tubli Bay,217315,en,wdpa-oecm-feature-data-layer


In [27]:
lookup.to_csv('/Users/sofia/Documents/HE_Data/Precalculated/AOI_lookup_table.csv')

<a id='publish'></a>
## Publish AOI lookup table

In [26]:
# Function to publish files as feature services or hosted tables

def publishCSVasFS(csv_file,csvName, gis, aol_folder_name, sharing, table): # sharing = 'None', 'groups', 'everyone' # table = 'yes', 'no'
    try:
        
        if gis.content.is_service_name_available(csvName, "featureService"):
            print(f"Service name {csvName} is available")
            csv_item = gis.content.add({}, csv_file)
            if table == 'yes':
                csv_lyr = csv_item.publish(publish_parameters={'locationType':'none', 'type':'csv'})
                print('done')
            else:
                print('no table')
                csv_lyr = csv_item.publish()
                flayer_collection = FeatureLayerCollection.fromitem(csv_lyr)
                searched_flayer = flayer_collection.layers[0] 
                nber_features = searched_flayer.query(return_count_only=True)
                print(f"The service {csvName} has been published. The service has {nber_features} entries")
            print(f"Moving service {csvName} to {aol_folder_name} in ArcGIS Online...")
            # move both the csv_item and csv_lyr items into this new folder
            csv_item.move(aol_folder_name)
            csv_lyr.move(aol_folder_name)
            print(f"Service {csvName} has been moved to {aol_folder_name} in ArcGIS Online")
#             sharing, default is owner
#             csv_lyr.shared_with
            if sharing == 'None':
                csv_lyr.share(everyone=False, org=False, groups=None, allow_members_to_edit=False)
            if sharing == 'groups':
            # the group id is needed to share with agroup
                csv_lyr.share(everyone=False, org=False, groups=[group_id], allow_members_to_edit=False)
            if sharing == 'everyone':
                csv_lyr.share(everyone=True, org=False, groups=None, allow_members_to_edit=False)
            sharing_prop = csv_lyr.shared_with
            if sharing_prop['everyone']==True:
                print(f"shared with everyone")   
            else:
                print(f"not a public layer, for this layer to be used it has to be public or the urls have to be whitelisted")            
            #not allowing deleting
            csv_lyr.protect()
            print(f"{csvName}'s protection against deletion : {csv_lyr.protected}") 
            # it is possible to check the status with csv_item.content_status
            return csv_lyr.id
        else:
            print("The service name is not available, try overwritting, appending the data or a different service name")
    except Exception as e:
        print(e)

In [27]:
# Run function with the file we want to add
csv_file = '/Users/sofia/Documents/HE_Data/Precalculated/AOI_lookup.csv'
csvName = 'AOI_lookup'
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)
aol_folder_name = '#6 precalculated data'
group_id='46783c33c294485abe83f5e899c2f21f'
publishCSVasFS(csv_file,csvName,gis,aol_folder_name,sharing='None', table='yes')

Service name AOI_lookup is available
{'message': "Service name 'data' already exists for 'IkktFdUAcY3WrH25'"}


#### NOTE:
For some unknown reason, the command `.publish()` doesn't work. It gives an error saying that the service name *data* already exists, but we don't know where that name comes from (it's not defined anywhere, the csv table has another name). The same happens with other .csv files, so it must be a problem with the API. We'll discuss this with ESRI but, for now, the table has been published manually.

<a id='update'></a>
## Update AOI lookup table: Example

It can be that some of the precalculated tables change, either because we changed the name of the geometry or we added new geometries. To be able to integrate these changes in the AOI search functionality, we need to periodically update the AOI lookup table.

To provide an example of how the table can be updated, we have made 2 changes on the cloned [gadm0_precalculated](https://eowilson.maps.arcgis.com/home/item.html?id=b9be6604edb448f5a1e08c3d292bdf8d#data) table we are using for testing:

* *NAME_0:* `Spain` is now called `Republic of Spain`, 
* *alternative_name* has a couple of fake alternative names for Spain: `Hispania` (Roman name) and `Tierra de conejos` (meaning of Hispania). 

Let's create a new AOI looup table with this changes and update the hosted table. The best approach to update the AOI lookup table hosted in AGOL is to overwrite it. Updating fields and adding new rows as described [here](https://developers.arcgis.com/python/samples/updating-features-in-a-feature-layer/) is not efficient for this table, as it has to identify the changes row by row and adding new rows (something that can happen in the AOI lookup table if we incorporate new precalculated areas) is done as a different step. Therefore, the most efficient way to update the hosted table is to overwrite it. For that, we followed [this method](https://developers.arcgis.com/python/samples/overwriting-feature-layers/). 

**Import the AOI layers**

In [48]:
gadm0 = getGadm0fromId('b9be6604edb448f5a1e08c3d292bdf8d') # this already has the abovementioned changes
gadm1 = getGadm1fromId('fe214eeebd21493eb2782a7ce1466606')
wdpa = getPAfromId('be073a211bcb4c7b859e1cb77f64eff0') 

**Create new AOI lookup table**

In [49]:
# Import gadm0 names (original and alternative)
names0 = gadm0[['NAME_0', 'MOL_ID']]
names0 [names0.NAME_0=='Republic of Spain'] # New name already there

,NAME_0,MOL_ID
69,Republic of Spain,70


In [50]:
# Create table for alternative names
alternative = gadm0[['alternative_names', 'MOL_ID']].dropna()
alternative # The new fake alternative names for Spain are there

,alternative_names,MOL_ID
44,Ivory Coast,45
58,Czech Republic,59
69,"Hispania, Tierra de conejos",70
144,Republic of Macedonia,145
147,Burma,148
211,Eswatini,212
226,Türkiye,227
234,"US, United States of America, USA",235


In [51]:
# Convert string with commas into list of string and strip spaces
alternative['alternative_names'] = alternative['alternative_names'].str.split(',').map(lambda elements: [e.strip() for e in elements])
# Explode lists in the column 'alternative_names' into separate values and reset index 
alternative = alternative.explode('alternative_names').reset_index(drop=True)
# Change name to NAME_0
alternative = alternative.rename(columns={'alternative_names':'NAME_0'})
alternative

,NAME_0,MOL_ID
0,Ivory Coast,45
1,Czech Republic,59
2,Hispania,70
3,Tierra de conejos,70
4,Republic of Macedonia,145
5,Burma,148
6,Eswatini,212
7,Türkiye,227
8,US,235
9,United States of America,235


In [52]:
# Create table with the gadm0 names
names0 = pd.concat([names0, alternative], axis=0).sort_values(by=['MOL_ID']).reset_index(drop=True).rename(columns={'NAME_0':'NAME'})
# Add layerslug for gadm0 
layerslug0 = "gadm-0-admin-areas-feature-layer"
names0['LAYERSLUG'] = layerslug0
names0

,NAME,MOL_ID,LAYERSLUG
0,Aruba,1,gadm-0-admin-areas-feature-layer
1,Afghanistan,2,gadm-0-admin-areas-feature-layer
2,Angola,3,gadm-0-admin-areas-feature-layer
3,Anguilla,4,gadm-0-admin-areas-feature-layer
4,Åland,5,gadm-0-admin-areas-feature-layer
...,...,...,...
261,Spratly Islands,251,gadm-0-admin-areas-feature-layer
262,Yemen,252,gadm-0-admin-areas-feature-layer
263,South Africa,253,gadm-0-admin-areas-feature-layer
264,Zambia,254,gadm-0-admin-areas-feature-layer


In [53]:
# Import subnational boundaries (gadm1)
names1 = gadm1[['NAME_1', 'MOL_ID']].rename(columns = {'NAME_1':'NAME'})
# Add layerslug for gadm1
layerslug1 = "gadm-1-admin-areas-feature-layer"
names1['LAYERSLUG'] = layerslug1
names1

,NAME,MOL_ID,LAYERSLUG
0,Akkar,1597,gadm-1-admin-areas-feature-layer
1,Maluku,1194,gadm-1-admin-areas-feature-layer
2,Baalbak - Hermel,1598,gadm-1-admin-areas-feature-layer
3,Maluku Utara,1195,gadm-1-admin-areas-feature-layer
4,Cotopaxi,801,gadm-1-admin-areas-feature-layer
...,...,...,...
3605,Western Cape,3590,gadm-1-admin-areas-feature-layer
3606,Central,3591,gadm-1-admin-areas-feature-layer
3607,Copperbelt,3592,gadm-1-admin-areas-feature-layer
3608,Eastern,3593,gadm-1-admin-areas-feature-layer


In [54]:
# Import protected areas
wdpa = wdpa[['NAME', 'MOL_ID']]
# Add layerslug 
layerslug2 = "wdpa-oecm-feature-data-layer"
wdpa['LAYERSLUG'] = layerslug2
wdpa

,NAME,MOL_ID,LAYERSLUG
0,Mt Benger,151,wdpa-oecm-feature-data-layer
1,Boulder Beach,1,wdpa-oecm-feature-data-layer
2,Thomas Hunt & Son,101,wdpa-oecm-feature-data-layer
3,Frame (Dunbarton - Millers Flat),152,wdpa-oecm-feature-data-layer
4,Earnscleugh - Obelisk Face,153,wdpa-oecm-feature-data-layer
...,...,...,...
217481,Kelly and Lennie Lake,217428,wdpa-oecm-feature-data-layer
217482,Sam McCrae Lake,217429,wdpa-oecm-feature-data-layer
217483,Mountain River Extension,217430,wdpa-oecm-feature-data-layer
217484,Lac Des Bois,217431,wdpa-oecm-feature-data-layer


**AOI lookup table**

In [55]:
frames = [names0, names1, wdpa]
lookup = pd.concat(frames)
# lookup['ID'] = pd.Series(range(1,len(lookup)))
lookup

,NAME,MOL_ID,LAYERSLUG
0,Aruba,1,gadm-0-admin-areas-feature-layer
1,Afghanistan,2,gadm-0-admin-areas-feature-layer
2,Angola,3,gadm-0-admin-areas-feature-layer
3,Anguilla,4,gadm-0-admin-areas-feature-layer
4,Åland,5,gadm-0-admin-areas-feature-layer
...,...,...,...
217481,Kelly and Lennie Lake,217428,wdpa-oecm-feature-data-layer
217482,Sam McCrae Lake,217429,wdpa-oecm-feature-data-layer
217483,Mountain River Extension,217430,wdpa-oecm-feature-data-layer
217484,Lac Des Bois,217431,wdpa-oecm-feature-data-layer


In [56]:
# Save new lookup table in a different folder with the same name as the old file
path = '/Users/sofia/Documents/HE_Data/Precalculated'
outdir= '{0}/updating_gis_content'.format(path)
if not os.path.exists(outdir):
    os.makedirs(outdir)
lookup.to_csv(f'{path}/updating_gis_content/AOI_lookup.csv')

Now that we have a new lookup table that includes the new updates, we need to update the AOI lookup table hosted in ArcGIS Online.

**Overwrite Hosted table**

In [58]:
# Import hosted table
item_hosted = gis.content.get('a57d934fc9e742cda867d39bef9c54ec') # id of the published hosted table
flayer_hosted = item_hosted.tables[0]
hosted_fset = flayer_hosted.query() #querying without any conditions returns all the features
hosted_sdf = hosted_fset.sdf
hosted_sdf.head()

,NAME,MOL_ID,LAYERSLUG,ObjectId
0,Aruba,1,gadm-0-admin-areas-feature-layer,1
1,Afghanistan,2,gadm-0-admin-areas-feature-layer,2
2,Angola,3,gadm-0-admin-areas-feature-layer,3
3,Anguilla,4,gadm-0-admin-areas-feature-layer,4
4,Åland,5,gadm-0-admin-areas-feature-layer,5


In [59]:
hosted_sdf[(hosted_sdf.MOL_ID==70)&(hosted_sdf.LAYERSLUG=='gadm-0-admin-areas-feature-layer')] # Hosted table has older name and no alternative names

,NAME,MOL_ID,LAYERSLUG,ObjectId
171,Spain,70,gadm-0-admin-areas-feature-layer,172


In [60]:
# Overwrite hosted table with the updated version
lookup_flayer_collection = FeatureLayerCollection.fromitem(item_hosted)
lookup_flayer_collection.manager.overwrite(f'{path}/updating_gis_content/AOI_lookup.csv')

{'success': True}

In [61]:
# Import new hosted table to check that has been updated
item_hosted = gis.content.get('a57d934fc9e742cda867d39bef9c54ec') # id of the published hosted table
flayer_hosted = item_hosted.tables[0]
hosted_fset = flayer_hosted.query() #querying without any conditions returns all the features
hosted_sdf = hosted_fset.sdf

In [62]:
hosted_sdf[(hosted_sdf.MOL_ID==70) &(hosted_sdf.LAYERSLUG=='gadm-0-admin-areas-feature-layer')] # The new names are there

,NAME,MOL_ID,LAYERSLUG,ObjectId
121,Republic of Spain,70,gadm-0-admin-areas-feature-layer,122
122,Hispania,70,gadm-0-admin-areas-feature-layer,123
123,Tierra de conejos,70,gadm-0-admin-areas-feature-layer,124


Now that we have seen that the update method works, we'll rerun the first part of this notebook (excluding point 4) using the original [gadm0_precalculated layer](https://eowilson.maps.arcgis.com/home/item.html?id=f11bd71ecc4c47aab7735b3cf3e21392) to create the AOI look up table. For updating it periodically, we have a [hosted notebook](https://eowilson.maps.arcgis.com/home/notebook/notebook.html?id=ff695ca1b2dc48e7a0b6a9ab79614924#)in ArcGIS Online that automatically reruns once a month to integrate any possible changes.